In [2]:
# Import libraries
import os
import json
import ee
import geemap
import geopandas as gpd
import pandas as pd
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings('ignore')

In [3]:
# Load shapefile data from the Train directory
try:
    fergana_train = gpd.read_file('data/train/Fergana_training_samples.shp')
    orenburg_train = gpd.read_file('data/train/Orenburg_training_samples.shp')
    
    print("\nFergana training data shape:", fergana_train.shape)
    print("Orenburg training data shape:", orenburg_train.shape)
    
    print("\nFergana training data head:")
    display(fergana_train.head())
    
    print("\nOrenburg training data head:")
    display(orenburg_train.head())
except Exception as e:
    print(f"Error loading shapefile data: {e}")
    print("You may need to install geopandas: pip install geopandas")



Fergana training data shape: (500, 3)
Orenburg training data shape: (500, 3)

Fergana training data head:


,ID,Cropland,geometry
0,1,0,POINT (71.74908 41.39632)
1,2,0,POINT (72.90143 41.07219)
2,3,1,POINT (72.46399 40.61906)
3,4,0,POINT (72.19732 41.62759)
4,5,0,POINT (71.31204 41.36946)



Orenburg training data head:


,ID,Cropland,geometry
0,1,0,POINT (54.74908 51.39632)
1,2,0,POINT (55.90143 51.07219)
2,3,0,POINT (55.19732 51.62759)
3,4,0,POINT (54.31204 51.36946)
4,5,0,POINT (54.11617 51.82185)


In [7]:
# Initialize Earth Engine
service_account = 'ee-service-account@clean-framework-466111-t0.iam.gserviceaccount.com'
key_file = 'credentials.json'

credentials = ee.ServiceAccountCredentials(service_account, key_file)
ee.Initialize(credentials, project='clean-framework-466111-t0')

In [10]:
def prepare_training_points():
    """Load and prepare training points from shapefiles"""
    # Load training shapefiles
    fergana_train = gpd.read_file('data/train/Fergana_training_samples.shp')
    orenburg_train = gpd.read_file('data/train/Orenburg_training_samples.shp')
    
    # Add location column if not present
    if 'location' not in fergana_train.columns:
        fergana_train['location'] = 'Fergana'
    if 'location' not in orenburg_train.columns:
        orenburg_train['location'] = 'Orenburg'
    
    # Combine training data
    train_gdf = pd.concat([fergana_train, orenburg_train], ignore_index=True)
    
    # Create unique IDs if needed
    if 'ID' not in train_gdf.columns:
        train_gdf['ID'] = [f"TRAIN_{i}" for i in range(len(train_gdf))]
    
    # Verify coordinates are in the correct range for Fergana and Orenburg
    # Fergana, Uzbekistan is roughly around 40-42°N, 71-73°E
    # Orenburg, Russia is roughly around 51-52°N, 54-56°E
    
    print("Verifying coordinates...")
    for idx, row in train_gdf.iterrows():
        lon = row.geometry.x
        lat = row.geometry.y
        location = row['location']
        
        # Check if coordinates are in expected range
        if location == 'Fergana':
            if not (39 <= lat <= 43 and 70 <= lon <= 74):
                print(f"WARNING: Point {idx} has suspicious coordinates for Fergana: lat={lat}, lon={lon}")
        elif location == 'Orenburg':
            if not (50 <= lat <= 53 and 53 <= lon <= 57):
                print(f"WARNING: Point {idx} has suspicious coordinates for Orenburg: lat={lat}, lon={lon}")
    
    # Check if we need to reproject to WGS84 (EPSG:4326)
    if train_gdf.crs and train_gdf.crs != "EPSG:4326":
        print(f"Reprojecting from {train_gdf.crs} to EPSG:4326 (WGS84)")
        train_gdf = train_gdf.to_crs("EPSG:4326")
    
    return train_gdf

train_gdf = prepare_training_points()
train_gdf.head()

Verifying coordinates...


,ID,Cropland,geometry,location
0,1,0,POINT (71.74908 41.39632),Fergana
1,2,0,POINT (72.90143 41.07219),Fergana
2,3,1,POINT (72.46399 40.61906),Fergana
3,4,0,POINT (72.19732 41.62759),Fergana
4,5,0,POINT (71.31204 41.36946),Fergana


In [19]:
# Initialize Earth Engine with service account credentials
def initialize_ee():
    service_account = 'ee-service-account@clean-framework-466111-t0.iam.gserviceaccount.com'
    key_file = 'credentials.json'
    credentials = ee.ServiceAccountCredentials(service_account, key_file)
    ee.Initialize(credentials, project='clean-framework-466111-t0')
    print("Earth Engine initialized successfully")


def prepare_training_points():
    """Prepare training points from shapefiles or geojson"""
    # Try loading from geojson first
    if os.path.exists('training_points.geojson'):
        return gpd.read_file('training_points.geojson')
    
    # If geojson doesn't exist, load from shapefiles
    fergana_train = gpd.read_file('data/train/Fergana_training_samples.shp')
    orenburg_train = gpd.read_file('data/train/Orenburg_training_samples.shp')
    
    # Add location column
    fergana_train['location'] = 'Fergana'
    orenburg_train['location'] = 'Orenburg'
    
    # Combine datasets
    train_gdf = pd.concat([fergana_train, orenburg_train], ignore_index=True)
    
    # Save for future use
    train_gdf.to_file('training_points.geojson', driver='GeoJSON')
    
    return train_gdf


def extract_s1_data(point_geom, point_id, cropland_value, start_date, end_date):
    """Extract raw Sentinel-1 data for a point"""
    # Ensure we're using the correct coordinate order (longitude, latitude)
    lon = point_geom.x  # This should be longitude
    lat = point_geom.y  # This should be latitude
    
    # Create EE point with explicit longitude, latitude order
    point = ee.Geometry.Point([lon, lat])
    buffer = point.buffer(30)  # 30m buffer
    
    try:
        # Get Sentinel-1 collection - use more specific filters to get fewer but better images
        s1_collection = ee.ImageCollection('COPERNICUS/S1_GRD') \
            .filterBounds(buffer) \
            .filterDate(start_date, end_date) \
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
            .filter(ee.Filter.eq('instrumentMode', 'IW'))
        
        # Check if collection is empty
        count = s1_collection.size().getInfo()
        if count == 0:
            print(f"No S1 images found for point {point_id}")
            return []
        
        # Get only 5 images with best coverage
        count = min(count, 5)
        images = s1_collection.toList(count)
        
        results = []
        for i in range(count):
            try:
                image = ee.Image(images.get(i))
                
                # Get all metadata in a single call to reduce API requests
                metadata = image.getInfo()
                date = ee.Date(metadata['properties']['system:time_start']).format('YYYY-MM-dd').getInfo()
                
                # Extract orbit information from metadata
                orbit = metadata['properties'].get('orbitProperties_pass', 'UNKNOWN')
                rel_orbit = metadata['properties'].get('relativeOrbitNumber_start', None)
                
                # Get both VV and VH bands in a single call if possible
                bands_to_get = ['VV']
                if 'VH' in metadata['bands'][0]['id']:
                    bands_to_get.append('VH')
                
                # Get band values
                bands = image.select(bands_to_get).reduceRegion(
                    reducer=ee.Reducer.mean(),
                    geometry=buffer,
                    scale=10
                ).getInfo()
                
                vv = bands.get('VV')
                vh = bands.get('VH')
                
                results.append({
                    'ID': point_id,
                    'VH': vh,
                    'VV': vv,
                    'date': date,
                    'orbit': orbit,
                    'polarization': '["VV"]' if vh is None else '["VV", "VH"]',
                    'rel_orbit': rel_orbit,
                    'translated_lat': lat,
                    'translated_lon': lon,
                    'Cropland': cropland_value  # Add Cropland target column
                })
            except Exception as e:
                print(f"Error processing S1 image {i} for point {point_id}: {e}")
        
        return results
    except Exception as e:
        print(f"Error in S1 extraction for point {point_id}: {e}")
        return []


def extract_s2_data(point_geom, point_id, cropland_value, start_date, end_date):
    """Extract raw Sentinel-2 data for a point"""
    # Get coordinates
    lon = point_geom.x
    lat = point_geom.y
    
    # Create EE point
    point = ee.Geometry.Point([lon, lat])
    buffer = point.buffer(30)  # 30m buffer
    
    try:
        # Get Sentinel-2 collection - use more specific filters to get better quality images
        s2_collection = ee.ImageCollection('COPERNICUS/S2_SR') \
            .filterBounds(buffer) \
            .filterDate(start_date, end_date) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
            .limit(5)  # Limit to 5 images to improve performance
        
        # Check if collection is empty
        count = s2_collection.size().getInfo()
        if count == 0:
            # Try again with higher cloud percentage if no images found
            s2_collection = ee.ImageCollection('COPERNICUS/S2_SR') \
                .filterBounds(buffer) \
                .filterDate(start_date, end_date) \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 80)) \
                .limit(3)  # Just get a few images
            count = s2_collection.size().getInfo()
            
            if count == 0:
                print(f"No S2 images found for point {point_id}")
                return []
        
        # Get list of images
        images = s2_collection.toList(count)
        
        results = []
        for i in range(count):
            try:
                image = ee.Image(images.get(i))
                
                # Get all metadata in a single call
                metadata = image.getInfo()
                date = ee.Date(metadata['properties']['system:time_start']).format('YYYY-MM-dd').getInfo()
                
                # Get cloud percentage and solar angles from metadata
                cloud_pct = metadata['properties'].get('CLOUDY_PIXEL_PERCENTAGE')
                solar_azimuth = metadata['properties'].get('MEAN_SOLAR_AZIMUTH_ANGLE')
                solar_zenith = metadata['properties'].get('MEAN_SOLAR_ZENITH_ANGLE')
                
                # Get all bands in a single call
                bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12']
                values = image.select(bands).reduceRegion(
                    reducer=ee.Reducer.mean(),
                    geometry=buffer,
                    scale=20  # Use 20m scale instead of 10m for faster processing
                ).getInfo()
                
                results.append({
                    'ID': point_id,
                    'B2': values.get('B2'),
                    'B3': values.get('B3'),
                    'B4': values.get('B4'),
                    'B5': values.get('B5'),
                    'B6': values.get('B6'),
                    'B7': values.get('B7'),
                    'B8': values.get('B8'),
                    'B8A': values.get('B8A'),
                    'B11': values.get('B11'),
                    'B12': values.get('B12'),
                    'cloud_pct': cloud_pct,
                    'date': date,
                    'solar_azimuth': solar_azimuth,
                    'solar_zenith': solar_zenith,
                    'translated_lat': lat,
                    'translated_lon': lon,
                    'Cropland': cropland_value  # Add Cropland target column
                })
            except Exception as e:
                print(f"Error processing S2 image {i} for point {point_id}: {e}")
        
        return results
    except Exception as e:
        print(f"Error in S2 extraction for point {point_id}: {e}")
        return []


def process_point(row):
    """Process a single point to extract S1 and S2 data"""
    point_id = row['ID']
    point_geom = row.geometry
    cropland_value = row['Cropland']  # Get Cropland value from the row
    
    # Use a single time period for the whole year to reduce API calls
    # We'll just get the best images from the entire year
    start_date = '2020-01-01'
    end_date = '2020-12-31'
    
    print(f"Processing point {point_id} (Cropland={cropland_value}) for period {start_date} to {end_date}")
    
    s1_results = []
    s2_results = []
    
    try:
        # Extract Sentinel-1 data for the whole year
        s1_batch = extract_s1_data(point_geom, point_id, cropland_value, start_date, end_date)
        if s1_batch:
            s1_results.extend(s1_batch)
            print(f"Added {len(s1_batch)} S1 images for point {point_id}")
        
        # Extract Sentinel-2 data for the whole year
        s2_batch = extract_s2_data(point_geom, point_id, cropland_value, start_date, end_date)
        if s2_batch:
            s2_results.extend(s2_batch)
            print(f"Added {len(s2_batch)} S2 images for point {point_id}")
        
        return {'s1': s1_results, 's2': s2_results, 'point_id': point_id}
    except Exception as e:
        print(f"Error processing point {point_id}: {e}")
        return {'s1': [], 's2': [], 'point_id': point_id}


def main():
    """Main execution function"""
    # Initialize Earth Engine
    initialize_ee()
    
    print("Preparing training points...")
    
    # Load all training points
    train_gdf = prepare_training_points()
    print(f"Loaded {len(train_gdf)} training points")
    
    # Read column structure from existing files
    s1_columns = pd.read_csv('data/Sentinel1.csv', nrows=0).columns.tolist()
    s2_columns = pd.read_csv('data/Sentinel2.csv', nrows=0).columns.tolist()
    
    # Add Cropland column if not already present
    if 'Cropland' not in s1_columns:
        s1_columns.append('Cropland')
    if 'Cropland' not in s2_columns:
        s2_columns.append('Cropland')
    
    print("S1 columns:", s1_columns)
    print("S2 columns:", s2_columns)
    
    # Create output directory if it doesn't exist
    os.makedirs('extracted_data', exist_ok=True)
    
    # Process in batches to avoid memory issues
    batch_size = 50
    num_batches = (len(train_gdf) + batch_size - 1) // batch_size
    
    for batch_idx in range(num_batches):
        start_idx = batch_idx * batch_size
        end_idx = min((batch_idx + 1) * batch_size, len(train_gdf))
        
        print(f"\nProcessing batch {batch_idx+1}/{num_batches} (points {start_idx+1}-{end_idx})")
        
        batch_gdf = train_gdf.iloc[start_idx:end_idx]
        s1_results = []
        s2_results = []
        
        # Process points in parallel with ThreadPoolExecutor
        max_workers = 8  # Adjust based on your system capabilities and API rate limits
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            # Submit all tasks
            future_to_idx = {executor.submit(process_point, row): idx for idx, row in batch_gdf.iterrows()}
            
            # Process results as they complete with tqdm
            for future in tqdm(as_completed(future_to_idx), total=len(future_to_idx), desc=f"Batch {batch_idx+1}"):
                idx = future_to_idx[future]
                try:
                    result = future.result()
                    s1_results.extend(result['s1'])
                    s2_results.extend(result['s2'])
                    
                    # Print summary without breaking tqdm
                    tqdm.write(f"Point {result['point_id']}: {len(result['s1'])} S1 images, {len(result['s2'])} S2 images")
                except Exception as e:
                    point_id = batch_gdf.iloc[idx-batch_gdf.index[0]]['ID'] if 'ID' in batch_gdf.columns else f"Point_{idx}"
                    tqdm.write(f"Error processing point {point_id}: {e}")
        
        # Convert to DataFrames
        s1_batch_df = pd.DataFrame(s1_results) if s1_results else pd.DataFrame(columns=s1_columns)
        s2_batch_df = pd.DataFrame(s2_results) if s2_results else pd.DataFrame(columns=s2_columns)
        
        # Ensure columns match expected structure
        for col in s1_columns:
            if col not in s1_batch_df.columns:
                s1_batch_df[col] = None
        
        for col in s2_columns:
            if col not in s2_batch_df.columns:
                s2_batch_df[col] = None
        
        # Keep only columns that match the expected structure
        s1_batch_df = s1_batch_df[s1_columns]
        s2_batch_df = s2_batch_df[s2_columns]
        
        # Save batch data
        s1_batch_df.to_csv(f'extracted_data/train_sentinel1_batch_{batch_idx+1}.csv', index=False)
        s2_batch_df.to_csv(f'extracted_data/train_sentinel2_batch_{batch_idx+1}.csv', index=False)
        
        print(f"Batch {batch_idx+1} complete! Data shapes:")
        print(f"Sentinel-1: {s1_batch_df.shape}")
        print(f"Sentinel-2: {s2_batch_df.shape}")
    
    # Combine all batches
    print("\nCombining all batches...")
    
    s1_files = [f for f in os.listdir('extracted_data') if f.startswith('train_sentinel1_batch_')]
    s2_files = [f for f in os.listdir('extracted_data') if f.startswith('train_sentinel2_batch_')]
    
    s1_dfs = [pd.read_csv(os.path.join('extracted_data', f)) for f in s1_files]
    s2_dfs = [pd.read_csv(os.path.join('extracted_data', f)) for f in s2_files]
    
    s1_train_df = pd.concat(s1_dfs, ignore_index=True) if s1_dfs else pd.DataFrame(columns=s1_columns)
    s2_train_df = pd.concat(s2_dfs, ignore_index=True) if s2_dfs else pd.DataFrame(columns=s2_columns)
    
    # Save combined data
    s1_train_df.to_csv('train_sentinel1_full.csv', index=False)
    s2_train_df.to_csv('train_sentinel2_full.csv', index=False)
    
    print("\nExtraction complete! Final data shapes:")
    print(f"Sentinel-1: {s1_train_df.shape}")
    print(f"Sentinel-2: {s2_train_df.shape}")
    
    # Print Cropland distribution
    if 'Cropland' in s1_train_df.columns:
        print("\nCropland distribution in Sentinel-1 data:")
        print(s1_train_df['Cropland'].value_counts())
    
    if 'Cropland' in s2_train_df.columns:
        print("\nCropland distribution in Sentinel-2 data:")
        print(s2_train_df['Cropland'].value_counts())
    
    return s1_train_df, s2_train_df

In [20]:
s1_train_df, s2_train_df = main()

Earth Engine initialized successfully
Preparing training points...
Loaded 1000 training points
S1 columns: ['ID', 'VH', 'VV', 'date', 'orbit', 'polarization', 'rel_orbit', 'translated_lat', 'translated_lon', 'Cropland']
S2 columns: ['B11', 'B12', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'ID', 'cloud_pct', 'date', 'solar_azimuth', 'solar_zenith', 'translated_lat', 'translated_lon', 'Cropland']

Processing batch 1/20 (points 1-50)
Processing point 1 (Cropland=0) for period 2020-01-01 to 2020-12-31
Processing point 2 (Cropland=0) for period 2020-01-01 to 2020-12-31
Processing point 3 (Cropland=1) for period 2020-01-01 to 2020-12-31
Processing point 4 (Cropland=0) for period 2020-01-01 to 2020-12-31
Processing point 5 (Cropland=0) for period 2020-01-01 to 2020-12-31
Processing point 6 (Cropland=0) for period 2020-01-01 to 2020-12-31
Processing point 7 (Cropland=0) for period 2020-01-01 to 2020-12-31
Processing point 8 (Cropland=1) for period 2020-01-01 to 2020-12-31


Batch 1:   0%|          | 0/50 [00:00<?, ?it/s]

Added 5 S1 images for point 1
Added 5 S1 images for point 5


c:\Users\janen\anaconda3\envs\ml_env\Lib\site-packages\ee\deprecation.py:209: DeprecationWarning: 

Attention required for COPERNICUS/S2_SR! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR

  warnings.warn(warning, category=DeprecationWarning)


Added 5 S1 images for point 3
Added 5 S1 images for point 2
Added 5 S1 images for point 4
Added 5 S1 images for point 7
Added 5 S1 images for point 8
Added 5 S1 images for point 6
Added 5 S2 images for point 1
Processing point 9 (Cropland=0) for period 2020-01-01 to 2020-12-31
Point 1: 5 S1 images, 5 S2 images
Added 5 S2 images for point 4
Processing point 10 (Cropland=0) for period 2020-01-01 to 2020-12-31
Point 4: 5 S1 images, 5 S2 images
Added 5 S2 images for point 2
Processing point 11 (Cropland=0) for period 2020-01-01 to 2020-12-31
Point 2: 5 S1 images, 5 S2 images
Added 5 S2 images for point 3
Processing point 12 (Cropland=0) for period 2020-01-01 to 2020-12-31
Point 3: 5 S1 images, 5 S2 images
Added 5 S2 images for point 5
Processing point 13 (Cropland=0) for period 2020-01-01 to 2020-12-31
Point 5: 5 S1 images, 5 S2 images
Added 5 S2 images for point 7
Processing point 14 (Cropland=0) for period 2020-01-01 to 2020-12-31
Point 7: 5 S1 images, 5 S2 images
Added 5 S2 images for p

...

Added 5 S1 images for point 352
Added 5 S1 images for point 354
Added 5 S1 images for point 353
Added 5 S1 images for point 351
Added 5 S1 images for point 355
Added 5 S1 images for point 358
Added 5 S1 images for point 357
Added 5 S1 images for point 356
Added 5 S2 images for point 352
Processing point 359 (Cropland=0) for period 2020-01-01 to 2020-12-31
Point 352: 5 S1 images, 5 S2 images
Added 5 S2 images for point 354
Processing point 360 (Cropland=0) for period 2020-01-01 to 2020-12-31
Point 354: 5 S1 images, 5 S2 images
Added 5 S2 images for point 353
Processing point 361 (Cropland=1) for period 2020-01-01 to 2020-12-31
Point 353: 5 S1 images, 5 S2 images
Added 5 S2 images for point 351
Processing point 362 (Cropland=1) for period 2020-01-01 to 2020-12-31
Point 351: 5 S1 images, 5 S2 images
Added 5 S2 images for point 358
Processing point 363 (Cropland=1) for period 2020-01-01 to 2020-12-31
Point 358: 5 S1 images, 5 S2 images
Added 5 S2 images for point 355
Processing point 364 (

Batch 19:   0%|          | 0/50 [00:00<?, ?it/s]

Added 5 S1 images for point 401
Added 5 S1 images for point 407
Added 5 S1 images for point 403
Added 5 S1 images for point 405
Added 5 S1 images for point 406
Added 5 S1 images for point 402
Added 5 S1 images for point 404
Added 5 S1 images for point 408
Added 5 S2 images for point 401
Processing point 409 (Cropland=1) for period 2020-01-01 to 2020-12-31
Point 401: 5 S1 images, 5 S2 images
Added 5 S2 images for point 407
Processing point 410 (Cropland=1) for period 2020-01-01 to 2020-12-31
Point 407: 5 S1 images, 5 S2 images
Added 5 S2 images for point 406
Processing point 411 (Cropland=1) for period 2020-01-01 to 2020-12-31
Point 406: 5 S1 images, 5 S2 images
Added 5 S2 images for point 403
Processing point 412 (Cropland=1) for period 2020-01-01 to 2020-12-31
Point 403: 5 S1 images, 5 S2 images
Added 5 S2 images for point 402
Processing point 413 (Cropland=0) for period 2020-01-01 to 2020-12-31
Point 402: 5 S1 images, 5 S2 images
Added 5 S2 images for point 404
Processing point 414 (

Batch 20:   0%|          | 0/50 [00:00<?, ?it/s]

Added 5 S1 images for point 455
Added 5 S1 images for point 454
Added 5 S1 images for point 457
Added 5 S1 images for point 453
Added 5 S1 images for point 451
Added 5 S1 images for point 456
Added 5 S1 images for point 458
Added 5 S1 images for point 452
Added 5 S2 images for point 456
Processing point 459 (Cropland=1) for period 2020-01-01 to 2020-12-31
Point 456: 5 S1 images, 5 S2 images
Added 5 S2 images for point 457
Processing point 460 (Cropland=1) for period 2020-01-01 to 2020-12-31
Point 457: 5 S1 images, 5 S2 images
Added 5 S2 images for point 451
Processing point 461 (Cropland=1) for period 2020-01-01 to 2020-12-31
Point 451: 5 S1 images, 5 S2 images
Added 5 S2 images for point 454
Processing point 462 (Cropland=1) for period 2020-01-01 to 2020-12-31
Point 454: 5 S1 images, 5 S2 images
Added 5 S2 images for point 455
Processing point 463 (Cropland=1) for period 2020-01-01 to 2020-12-31
Point 455: 5 S1 images, 5 S2 images
Added 5 S2 images for point 458
Processing point 464 (

In [22]:
s1_train_df.head()

,ID,VH,VV,date,orbit,polarization,rel_orbit,translated_lat,translated_lon,Cropland
0,1,NaN,-10.608877,2020-01-01,DESCENDING,"[""VV""]",78,41.396323,71.74908,0
1,1,NaN,-12.619320,2020-01-02,ASCENDING,"[""VV""]",100,41.396323,71.74908,0
2,1,NaN,-13.819860,2020-01-08,DESCENDING,"[""VV""]",5,41.396323,71.74908,0
3,1,NaN,-10.964458,2020-01-13,DESCENDING,"[""VV""]",78,41.396323,71.74908,0
4,1,NaN,-12.817734,2020-01-14,ASCENDING,"[""VV""]",100,41.396323,71.74908,0


In [ ]:
s1_train_df.shape

(500, 10)